[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-pytorch/blob/paddle/demo.ipynb)

In [ ]:
%cd /content
!git clone -b paddle https://github.com/camenduru/stable-diffusion-pytorch

In [ ]:
!apt -y install -qq aria2
%cd /content/stable-diffusion-pytorch
%pip install -r requirements.txt

In [ ]:
%cd /content/stable-diffusion-pytorch
# !wget https://huggingface.co/jinseokim/stable-diffusion-pytorch-data/resolve/main/data.v20221029.tar
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/jinseokim/stable-diffusion-pytorch-data/resolve/main/data.v20221029.tar -d /content/stable-diffusion-pytorch -o data.v20221029.tar
!tar -xf data.v20221029.tar

In [ ]:
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://passl.bj.bcebos.com/models/ViT-B-32.pdparams -d /content/stable-diffusion-pytorch/data/ckpt -o clip.pt

In [ ]:
from stable_diffusion_pytorch import model_loader
models = model_loader.preload_models('cpu')

In [ ]:
from stable_diffusion_pytorch import pipeline

prompt = "a photograph of an astronaut riding a horse"  #@param { type: "string" }
prompts = [prompt]

uncond_prompt = ""  #@param { type: "string" }
uncond_prompts = [uncond_prompt] if uncond_prompt else None

upload_input_image = False  #@param { type: "boolean" }
input_images = None
if upload_input_image:
    from PIL import Image
    from google.colab import files
    print("Upload an input image:")
    path = list(files.upload().keys())[0]
    input_images = [Image.open(path)]

strength = 0.8  #@param { type:"slider", min: 0, max: 1, step: 0.01 }

do_cfg = True  #@param { type: "boolean" }
cfg_scale = 7.5  #@param { type:"slider", min: 1, max: 14, step: 0.5 }
height = 512  #@param { type: "integer" }
width = 512  #@param { type: "integer" }
sampler = "k_lms"  #@param ["k_lms", "k_euler", "k_euler_ancestral"]
n_inference_steps = 50  #@param { type: "integer" }

use_seed = False  #@param { type: "boolean" }
if use_seed:
    seed = 42  #@param { type: "integer" }
else:
    seed = None

pipeline.generate(prompts=prompts, uncond_prompts=uncond_prompts,
                  input_images=input_images, strength=strength,
                  do_cfg=do_cfg, cfg_scale=cfg_scale,
                  height=height, width=width, sampler=sampler,
                  n_inference_steps=n_inference_steps, seed=seed,
                  models=models, device='cuda', idle_device='cpu')[0]